In [1]:
import sys
sys.path.append('../src')

In [2]:
import warnings
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import Compose
from torch.utils.data import DataLoader

import augment
import model
from AortaDataset import AortaDataset

warnings.filterwarnings('ignore')

../src/model.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.conv.weight, gain=np.sqrt(2))
/Users/irrwitz/anaconda/envs/deep/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

cpu


In [4]:
net = model.net
net.to(device)

UNet(
  (down1): Sequential(
    (0): conv_block(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.01)
    )
    (1): conv_block(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.01)
    )
  )
  (down2): Sequential(
    (0): conv_block(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.01)
    )
    (1): conv_block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tr

In [5]:
def dice(pred, target):
    pred = (pred>0).float()
    return 2. * (pred*target).sum() / (pred+target).sum()

In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

In [7]:
flipud = augment.FlipUD()
fliplr = augment.FlipLR()
crop = augment.RandomCrop()
resize = augment.Resize(512, 388)
composed = Compose([flipud, fliplr, crop, resize])

In [8]:
aorta_train_dataset = AortaDataset('train', transform=composed)
aorta_test_dataset = AortaDataset('test', transform=composed)
aorta_train_loader = DataLoader(aorta_train_dataset, batch_size=3, shuffle=False)

In [9]:
print(len(aorta_train_dataset), len(aorta_test_dataset))

98 3


In [10]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(aorta_train_loader, 0):
        # get the inputs
        images, masks = data
        
        images = torch.unsqueeze(images,1).float().to(device)
        masks = masks.float().to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
print('Finished Training')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


ValueError: Target size (torch.Size([3, 388, 388])) must be the same as input size (torch.Size([3, 1, 512, 512]))

In [ ]:
torch.save(net.state_dict(), 'first_model.pt')